# testAPI_automate

Created by Davy Yue 2017-06-26

## Testing for Conversion into Function/Class ==============================

Code May Break At Anytime

In [1]:
# 2017_06_26 4:00 PM CST
# **** DEBUGGING/TESTING NOTES ****
# - wrap above code in a function
# - print the plot in one line
# - think about inputs and outputs
# - think about where variables can be passed as parameters
# - need test cases - maybe with n-dodecane
# - check with generated rdfs from above to compare results
# - maybe use more functions to call
# - consider adaptation for other organic molecules 
#      (carbon, hydrogen, nitrogen, oxygen, phosphorus, and sulfur)
#      CHNOP
# - need ability to have multiple elements (later testing phase)

### Imports

In [2]:
import itertools
import string
import os

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from msibi import MSIBI, State, Pair, mie
import mdtraj as md

# check possible other packages to ease process

### Functions

In [3]:
def initialize(element_names):
    ## for loop to traverse element_names array for elements
    ## first test - only use for carbon (one element)
    ## then expand later
    ## need to allow for different types of elements together
    
    map_beads(element_names)

In [4]:
def map_beads(element_names):
    # Initialize atoms with elements
    from mdtraj.core import element
    list(t.top.atoms)[0].element = element.carbon # check element
    list(t.top.atoms)[0].element.mass
    for atom in t.top.atoms:
        atom.element = element.carbon # check element
    
    # Map the beads accordingly
    cg_idx = 0
    start_idx = 0
    ## maybe need another function to read files in and extract information
    ## then call the other funcitons with the necessary parameters
    n_propane = 1024 #passed later as parameter
    propane_map = {0: [0, 1, 2]} ## need better mapping definition
                                ## maybe integrate mapping with file read-in
    system_mapping = {}
    for n in range(n_propane):
        for bead, atoms in propane_map.items():
            system_mapping[cg_idx] = [x + start_idx for x in atoms]
            start_idx += len(atoms) # understand this part of the function
            cg_idx += 1    
    
    # Apply mapping for XYZ coordinates
    cg_xyz = np.empty((t.n_frames, len(system_mapping), 3))
    for cg_bead, aa_indices in system_mapping.items():
        cg_xyz[:, cg_bead, :] = md.compute_center_of_mass(t.atom_slice(aa_indices))
        
    # Apply mapping for Topology object
    cg_top = md.Topology()
    for cg_bead in system_mapping.keys(): #i got the keys keys keys
        cg_top.add_atom('carbon', element.virtual_site, cg_top.add_residue('A', 
                            cg_top.add_chain())) 
        ## Check element and name for items 'A'
        
    compute_files(cg_xyz, cg_top)

In [5]:
def compute_files(cg_xyz, cg_top):
    # Create Trajectory object and save to .dcd file 
    ## any file type might do - check with Tim
    ## maybe save to file type that is the same as inputted file-type
    cg_traj = md.Trajectory(cg_xyz, cg_top, time=None, 
                            unitcell_lengths=t.unitcell_lengths, 
                            unitcell_angles=t.unitcell_angles)
    cg_traj.save_dcd('cg_traj.dcd') ## need check statements to prevent file overwrite
                                    ## rename old/new files accordingly

    # Create rdfs file from pairs
    pairs = cg_traj.top.select_pairs(selection1='name "carbon"', 
                                     selection2='name "carbon"') # Check element
    r, g_r = md.compute_rdf(cg_traj, pairs=pairs, 
                            r_range=(0, 1.2), bin_width=0.005) 
                    ## identify end of range with data pairs
                    ## maybe something with read-file function - compare data values next to each other
                    ## See where data drop-off occurs and plot respectively
    np.savetxt('rdfs_aa.txt', np.transpose([r, g_r])) # need check statements to prevent file overwrite
    
    plot_output(r, g_r)

In [6]:
def plot_output(x, y):
    ## modify figsize according to drowpoff point
    plt.figure(num=None, figsize=(16, 12), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(x, y, label="CG Propane") ## Check label use string format method
    
    ## look up more aesthetic matplotlib functions
    plt.xlabel("r")
    plt.ylabel("g(r)")
    plt.legend()

In [7]:
def convert_Traj_RDF(element_names):
    ## add parameters to function calls - maybe add other functions
    ## other potential functions:
    ##     - check_file_overwrite()
    ##     - check_g_r_dropoff() - integrate with plot function
    ##     - account_for_different_file_types() - read in files
    
    initialize(element_names)

In [8]:
# Execute functions
element_names = ["carbon"]
traj_filename = 'traj_unwrapped.dcd'
struct_filename = 'start_aa.hoomdxml'
t = md.load(traj_filename, top=struct_filename)

convert_Traj_RDF(element_names)

NameError: name 'element' is not defined